# Open3D/Semantic Segmentation/Object detection

---



---



본 실습에서는 Open3D라이브러리를 이용하여 pointcloud를 활용하는 기본적인 방법과 Semantic Segmentation 및 Object detection pipeline을 inference하는 시간을 가지도록 하겠습니다.

보다 간단한 실습을 위해, 3D 데이터를 다루는 라이브러리인 Open3D의 Open3D-ML을 활용합니다.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


!which python


Mounted at /content/gdrive
/usr/local/bin/python


### Environment Setting

실습을 위한 필요한 라이브러리(PyTorch, Open3D)를 설치합니다.
기본적인 환경은 Python 3 환경에서 진행되며, 가능할 경우 Conda를 활용하여 환경을 구성하는 것을 추천드립니다.

In [1]:
# PyTorch with CUDA, CUDA driver version -> 11.6
# %pip install -r /content/gdrive/MyDrive/AI_Expert_0907_3DVision/requirements-torch-cuda.txt
%pip install -r /content/gdrive/MyDrive/ai_expert/32_3d_vision/requirements-torch-cuda.txt


Looking in links: https://download.pytorch.org/whl/torch/, https://download.pytorch.org/whl/torchvision/


In [2]:
%pip install --upgrade open3d
!python -c "import open3d.ml.torch as ml3d"


2023-09-07 07:06:23.896165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 07:06:24.889595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##1. Tensor creation

Open3D에서는 tensor로 바꾸는 것이 가능합니다.
기존에 학습하는 tensor를 open3d로 바꿔서 연산이 가능합니다.

In [3]:
import open3d as o3d
import open3d.core as o3c
import numpy as np

# Tensor from list.
a = o3c.Tensor([0, 1, 2])
print("Created from list:\n{}".format(a))

# Tensor from Numpy.
a = o3c.Tensor(np.array([0, 1, 2]))
print("\nCreated from numpy array:\n{}".format(a))

# Dtype and inferred from list.
a_float = o3c.Tensor([0.0, 1.0, 2.0])
print("\nDefault dtype and device:\n{}".format(a_float))

# Specify dtype.
a = o3c.Tensor(np.array([0, 1, 2]), dtype=o3c.Dtype.Float64)
print("\nSpecified data type:\n{}".format(a))

# Specify device.
a = o3c.Tensor(np.array([0, 1, 2]), device=o3c.Device("CPU:0"))
print("\nSpecified device:\n{}".format(a))

Created from list:
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x58ae1a0b4630]

Created from numpy array:
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x58ae1a1c7300]

Default dtype and device:
[0 1 2]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x58ae1a2eaf60]

Specified data type:
[0 1 2]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x58ae1a2d4200]

Specified device:
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x58ae1a1672b0]


## 2. Tensor properties
파이토치와 같이 다음과 같은 특성들을 가집니다.

In [4]:
vals = np.array((range(24))).reshape(2, 3, 4)
a = o3c.Tensor(vals, dtype=o3c.Dtype.Float64, device=o3c.Device("CPU:0"))
print(f"a.shape: {a.shape}")
print(f"a.strides: {a.strides}")
print(f"a.dtype: {a.dtype}")
print(f"a.device: {a.device}")
print(f"a.ndim: {a.ndim}")

a.shape: SizeVector[2, 3, 4]
a.strides: SizeVector[12, 4, 1]
a.dtype: Float64
a.device: CPU:0
a.ndim: 3


## 3. Tensor Copy & Device transfer

cpu와 gpu할당을 따로 해줄 수 있습니다.

In [5]:
a_cpu = o3c.Tensor([0, 1, 2])
a_gpu = a_cpu.cuda(0)
print(a_gpu)

# Device -> Host.
a_gpu = o3c.Tensor([0, 1, 2], device=o3c.Device("CUDA:0"))
a_cpu = a_gpu.cpu()
print(a_cpu)

# Device -> another Device.
a_gpu_0 = o3c.Tensor([0, 1, 2], device=o3c.Device("CUDA:0"))
a_gpu_1 = a_gpu_0.cuda(0)
print(a_gpu_1)

[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CUDA:0, 0x302000000]
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x58ae2eacc860]
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CUDA:0, 0x302000400]


##4. Data types.

데이터 타입도 따로 설정이 가능합니다.

In [6]:
# E.g. float -> int
a = o3c.Tensor([0.1, 1.5, 2.7])
b = a.to(o3c.Dtype.Int32)
print(a)
print(b)

[0.1 1.5 2.7]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x58ae2ee21e90]
[0 1 2]
Tensor[shape={3}, stride={1}, Int32, CPU:0, 0x58ae1acff360]


##5. Conversion from pytorch's tensor

Pytorch 텐서로 다음과 같이 변환이 용이합니다.

In [7]:
import torch
import torch.utils.dlpack

# From PyTorch
th_a = torch.ones((5,)).cuda(0)
o3_a = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

# To PyTorch
o3_a = o3c.Tensor([1, 1, 1, 1, 1], device=o3c.Device("CUDA:0"))
th_a = torch.utils.dlpack.from_dlpack(o3_a.to_dlpack())
o3_a = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

th_a: tensor([1., 1., 1., 1., 1.], device='cuda:0')
o3_a: [1 1 1 1 1]
Tensor[shape={5}, stride={1}, Float32, CUDA:0, 0x7b1975200000]

th_a: tensor([100., 200.,   1.,   1.,   1.], device='cuda:0')
o3_a: [100 200 1 1 1]
Tensor[shape={5}, stride={1}, Float32, CUDA:0, 0x7b1975200000]
th_a: tensor([1, 1, 1, 1, 1], device='cuda:0')
o3_a: [1 1 1 1 1]
Tensor[shape={5}, stride={1}, Int64, CUDA:0, 0x302000600]

th_a: tensor([100, 200,   1,   1,   1], device='cuda:0')
o3_a: [100 200 1 1 1]
Tensor[shape={5}, stride={1}, Int64, CUDA:0, 0x302000600]


##6. Binary element-wise operation

다양한 연산이 가능합니다.

In [8]:
a = o3c.Tensor([1, 1, 1], dtype=o3c.Dtype.Float32)
b = o3c.Tensor([2, 2, 2], dtype=o3c.Dtype.Float32)
print("a + b = {}".format(a + b))
print("a - b = {}".format(a - b))
print("a * b = {}".format(a * b))
print("a / b = {}".format(a / b))

a + b = [3 3 3]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x58ae72f910e0]
a - b = [-1 -1 -1]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x58ae34950fa0]
a * b = [2 2 2]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x58ae72f910e0]
a / b = [0.5 0.5 0.5]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x58ae34950fa0]


##7. Read PointCloud and write

예시 pointcloud를 불러와 다음과 같이 읽을 수 있습니다.

기존 code에서는 draw_geometries함수를, colab이나 jupyter에서는 draw_plotly를 이용합니다.

다음으로 pcd를 다음과 같이 저장합니다.


In [9]:
import open3d as o3d
import numpy as np
ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud(ply_point_cloud.path)
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_plotly([pcd])
o3d.io.write_point_cloud('./pointcloud.ply', pcd)


Output hidden; open in https://colab.research.google.com to view.

##8. Paint PointCloud
Pointcloud의 색상도 정할 수 있습니다.

In [10]:
print(np.asarray(pcd.colors))
pcd.paint_uniform_color([1, 0.706, 0])

print(np.asarray(pcd.colors))

o3d.visualization.draw_plotly([pcd])

Output hidden; open in https://colab.research.google.com to view.

Question 1

직접 pointcloud를 불러본 후 pointcloud의 색상을 다음과 같이 칠해봅시다.

In [11]:
import numpy as np
# points = np.fromfile('/content/gdrive/MyDrive/AI_Expert_0907_3DVision/SemanticKITTI_mini/dataset/sequences/00/velodyne/000001.bin', dtype=np.float32).reshape(-1,4)
# labels = np.fromfile('/content/gdrive/MyDrive/AI_Expert_0907_3DVision/SemanticKITTI_mini/dataset/sequences/00/labels/000001.label', dtype=np.int32).reshape(-1,1)
points = np.fromfile('/content/gdrive/MyDrive/ai_expert/32_3d_vision/SemanticKITTI_mini/dataset/sequences/00/velodyne/000001.bin', dtype=np.float32).reshape(-1,4)
labels = np.fromfile('/content/gdrive/MyDrive/ai_expert/32_3d_vision/SemanticKITTI_mini/dataset/sequences/00/labels/000001.label', dtype=np.int32).reshape(-1,1)

print(points.shape)
coords = points[:,:3]
print(labels.shape)
print(np.unique(labels))

def colormap(coords, labels):

  valid = labels.flatten() < 65536
  coords = coords[valid,:3]
  labels = labels[valid,:]
  map = {0:0, 1:1, 40:2, 44:3, 48:4, 50:6, 51:7, 52:8, 60:9, 70:10, 71:11, 72:12, 80:13, 81:14, 99:15}
  map_int = np.vectorize(map.__getitem__)(labels.flatten())

  return map_int, coords, labels

def semantic_colormap(labels):
  Colors = [[0., 0., 0.], [0.96078431, 0.58823529, 0.39215686],
              [0.96078431, 0.90196078, 0.39215686],
              [0.58823529, 0.23529412, 0.11764706],
              [0.70588235, 0.11764706, 0.31372549], [1., 0., 0.],
              [0.11764706, 0.11764706, 1.], [0.78431373, 0.15686275, 1.],
              [0.35294118, 0.11764706, 0.58823529], [1., 0., 1.],
              [1., 0.58823529, 1.], [0.29411765, 0., 0.29411765],
              [0.29411765, 0., 0.68627451], [0., 0.78431373, 1.],
              [0.19607843, 0.47058824, 1.], [0., 0.68627451, 0.],
              [0., 0.23529412,
               0.52941176], [0.31372549, 0.94117647, 0.58823529],
              [0.58823529, 0.94117647, 1.], [0., 0., 1.]]
  return np.array(Colors)[labels]

label_int, coords, _ = colormap(coords, labels)
colors = semantic_colormap(label_int)

print(colors)

pcd = o3d.geometry.PointCloud()

pcd.points = o3d.utility.Vector3dVector(coords)
pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_plotly([pcd])


Output hidden; open in https://colab.research.google.com to view.

## 데이터셋 읽기

SemanticKITTI 데이터셋을 다운 받아, Open3D-ML라이브러리로 읽어봅시다.

In [13]:
# Training Semantic Segmentation Model using PyTorch

# import torch
import open3d.ml.torch as ml3d

# Read a dataset by specifying the path. We are also providing the cache directory and training split.
dataset = ml3d.datasets.SemanticKITTI(dataset_path='/content/gdrive/MyDrive/ai_expert/32_3d_vision/SemanticKITTI_mini',
                                      cache_dir='logs/cache',
                                      training_split=['00'],
                                      validation_split=['01'],
                                      test_split=['01'])

# Split the dataset for 'training'. You can get the other splits by passing 'validation' or 'test'
train_split = dataset.get_split('training')
test_split = dataset.get_split('test')

#support of Open3d-ML visualizer in Jupyter Notebooks is in progress
#view the frames using the visualizer
#vis = ml3d.vis.Visualizer()
#vis.visualize_dataset(dataset, 'training',indices=range(len(train_split)))

Semantic Segmentation으로 활용할 RandLANet을 불러옵니다.
pipeline에는 학습하기 위한 dataset, max_epoch등을 설정하여 학습준비를 할 수 있습니다.

In [14]:
# Training Semantic Segmentation Model using PyTorch

# Import torch and the model to use for training
import open3d.ml.torch as ml3d
from open3d.ml.torch.models import RandLANet
from open3d.ml.torch.pipelines import SemanticSegmentation


# Initialize the RandLANet model.
model = RandLANet(in_channels=3)
pipeline = SemanticSegmentation(model=model,
                                dataset=dataset,
                                max_epoch=3,
                                optimizer={'lr': 0.001},
                                num_workers=0)

#pipeline.run_train()


미리 학습된 weight를 불러 다시 한번 test를 합니다.
data의 일부만 불러와서 inference를 진행합니다.

In [15]:
# download the weights.
import os

ckpt_folder = "pretrained_weights"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "/randlanet_semantickitti_202201071330utc.pth"
randlanet_url = "https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(randlanet_url, ckpt_path)
    os.system(cmd)

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

# Run the test
#pipeline.run_test()
train_split = dataset.get_split("training")
#test_split = dataset.get_split("test")
#data = test_split.get_data(0)
data = train_split.get_data(0)
# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

test 0/1: 100%|█████████▉| 83699/83891 [00:02<00:00, 29148.34it/s]

colormap을 이용해서 pointcloud를 시각화해봅시다.

In [16]:
# Visualization Semantic Segmentation results from Open3D-ML
import numpy as np
import open3d as o3d

#Question 1 : Visualize pointcloud
def semantic_colormap(labels):
  Colors = [[0., 0., 0.], [0.96078431, 0.58823529, 0.39215686],
              [0.96078431, 0.90196078, 0.39215686],
              [0.58823529, 0.23529412, 0.11764706],
              [0.70588235, 0.11764706, 0.31372549], [1., 0., 0.],
              [0.11764706, 0.11764706, 1.], [0.78431373, 0.15686275, 1.],
              [0.35294118, 0.11764706, 0.58823529], [1., 0., 1.],
              [1., 0.58823529, 1.], [0.29411765, 0., 0.29411765],
              [0.29411765, 0., 0.68627451], [0., 0.78431373, 1.],
              [0.19607843, 0.47058824, 1.], [0., 0.68627451, 0.],
              [0., 0.23529412,
               0.52941176], [0.31372549, 0.94117647, 0.58823529],
              [0.58823529, 0.94117647, 1.], [0., 0., 1.]]
  return np.array(Colors)[labels]

def o3d_get_pointcloud(coords, colors):
  pcd = o3d.geometry.PointCloud()
  v3d = o3d.utility.Vector3dVector
  pcd.points = v3d(coords)
  pcd.colors = v3d(colors)

  return pcd


def o3d_visualization_semseg(data, results):

  points = data['point']
  labels = data['label'].astype(np.int32)

  pred_labels = (results['predict_labels'] + 1).astype(np.int32)

  colors = semantic_colormap(labels)
  pred_colors = semantic_colormap(pred_labels)

  org_pcd = o3d_get_pointcloud(points, colors)
  pred_pcd = o3d_get_pointcloud(points, pred_colors)

  o3d.visualization.draw_plotly([org_pcd])
  o3d.visualization.draw_plotly([pred_pcd])

o3d_visualization_semseg(data, result)

Output hidden; open in https://colab.research.google.com to view.

## PointPillars with Open3D

물체검출 하는 모델인 PointPillar를 이용하여 KITTI 데이터셋을 확인합니다. Object detection 모델에서도 마찬가지로 모듈을 불러와서 pipeline을 제시합니다.

In [17]:
import os
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d

cfg_file = "/content/gdrive/MyDrive/ai_expert/32_3d_vision//configs/pointpillars_kitti.yml"

cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.PointPillars(**cfg.model, device='cuda:0')
cfg.dataset['dataset_path'] = "/content/gdrive/MyDrive/ai_expert/32_3d_vision//kitti_mini"
dataset = ml3d.datasets.KITTI(cfg.dataset.pop('dataset_path', None), **cfg.dataset)

# Change gpu to cpu if cuda is not available
pipeline = ml3d.pipelines.ObjectDetection(model, dataset=dataset, device="cuda:0", **cfg.pipeline)

print(pipeline)

#vis = ml3d.vis.Visualizer()
#vis.visualize_dataset(dataset, 'train', indices=range(100))

test 0/1: 100%|██████████| 83891/83891 [01:37<00:00, 859.06it/s]  


Question 2 : 이제 PointPillars 모델에 데이터를 통과시킨 뒤, 데이터가 어떤 방식으로 나오는지 알아봅시다.

In [18]:
from pprint import pprint
from open3d.ml.datasets import KITTI
import open3d.ml.torch as ml3d
from open3d.ml.torch.pipelines import ObjectDetection
from open3d.ml.vis import Visualizer, LabelLUT
import numpy as np
import os


# download the weights.
ckpt_folder = "pretrained_weights"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "/pointpillars_kitti_202012221652utc.pth"
pointpillar_url = "https://storage.googleapis.com/open3d-releases/model-zoo/pointpillars_kitti_202012221652utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(pointpillar_url, ckpt_path)
    os.system(cmd)

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

### train split, test split, data
#train_split = dataset.get_split('training')
test_split = dataset.get_split('test')
#data = train_split.get_data(0)
data = test_split.get_data(1)
result = pipeline.run_inference(data)

pipeline.run_test()

# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.

pprint(result)

for res in result[0]:
    res_dict = res.to_dict()
    print(f'bounding box {res_dict["label"]} x, y, z, l, w, h, theta:')
    print(res.to_xyzwhlr())

'''
def pred_custom_data(pc_names, pcs, pipeline):
    vis_points = []
    for i, data in enumerate(pcs):
        name = pc_names[i]
        pts = data['point']

        vis_d = {
            "name": name,
            "points": pts,
        }
        vis_points.append(vis_d)

    return vis_points
'''
# v = Visualizer()
# lut = LabelLUT()
# for val in sorted(kitti_labels.keys()):
#     lut.add_label(kitti_labels[val], val)
# v.set_lut("labels", lut)
# v.set_lut("bounding_boxes", lut)
# v.visualize(pcs_with_pred, bounding_boxes=results[0])
# evaluate performance on the test set; this will write logs to './logs'.


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning:

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:554: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

validation: 0it [00:00, ?it/s]
testing: 100%|██████████| 201/201 [01:29<00:00,  2.24it/s]

[[box:1 (class=Cyclist, conf=0.10098584),
  box:2 (class=Car, conf=0.9666018),
  box:3 (class=Car, conf=0.940865),
  box:4 (class=Car, conf=0.36341298),
  box:5 (class=Car, conf=0.18792629),
  box:6 (class=Car, conf=0.16548853),
  box:7 (class=Car, conf=0.1146354),
  box:8 (class=Car, conf=0.10387799)]]
bounding box Cyclist x, y, z, l, w, h, theta:
[21.41704369  8.16703415 -1.5199275   0.5904038   1.8623445   1.76353049
  1.53019893]
bounding box Car x, y, z, l, w, h, theta:
[17.66450119  8.64189816 -1.52344251  1.63222754  4.06144667  1.57162774
 -3.10736346]
bounding box Car x, y, z, l, w, h, theta:
[12.00894737  7.93859673 -1.38776064  1.65610409  4.05909872  1.55686092
 -2.47253156]
bounding box Car x, y, z, l, w, h, theta:
[16.79777145 13.69311619 -1.79524428  1.67516363  4.03367996  1.49377501
 -2.15967631]
bounding box Car x, y, z, l, w, h, theta:
[60.49819183 -4.68910551 -1.87466407  1.73205876  4.37716055  1.51198149
 -1.54930294]
bounding box Car x, y, z, l, w, h, theta:
[51.

'\ndef pred_custom_data(pc_names, pcs, pipeline):\n    vis_points = []\n    for i, data in enumerate(pcs):\n        name = pc_names[i]\n        pts = data[\'point\']\n\n        vis_d = {\n            "name": name,\n            "points": pts,\n        }\n        vis_points.append(vis_d)\n\n    return vis_points\n'

마지막으로 Bounding Box Visualization을 하는 방법을 확인해봅시다.(추가)

In [19]:
import open3d as o3d

def translate_boxes_to_open3d_instance(gt_boxes, label):
    """
             4-------- 6
           /|         /|
          5 -------- 3 .
          | |        | |
          . 7 -------- 1
          |/         |/
          2 -------- 0
    """

    color_map = {'Pedestrian':np.array([[0.,0.9,0.9]]), 'Cyclist':np.array([[0.9,0.9,0.]]), 'Car':np.array([[0.,0.9,0]])}
    color = color_map.__getitem__(label)


    ##Fill the blank##
    center = gt_boxes[0:3]
    lwh = gt_boxes[3:6]
    axis_angles = np.array([0, 0, gt_boxes[6]+1e-10])

    #center = gt_boxes[0:3]
    #lwh = gt_boxes[3:6]
    #axis_angles = np.array([0, 0, gt_boxes[6] + 1e-10])

    ##Use o3d geometry
    rot = o3d.geometry.get_rotation_matrix_from_axis_angle(axis_angles)
    box3d = o3d.geometry.OrientedBoundingBox(center, rot, lwh)
    line_set = o3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)

    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    colors = np.repeat(color,len(lines),0)

    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)

    return line_set, box3d

def o3d_get_pointcloud(coords, colors):
  pcd = o3d.geometry.PointCloud()
  v3d = o3d.utility.Vector3dVector
  pcd.points = v3d(coords)
  pcd.colors = v3d(colors)

  return pcd

#hack to better visualization
data['point'] = np.concatenate([data['point'], np.array([[0,0,30,0]])], axis=0)

pcd = o3d_get_pointcloud(data['point'][:,:3], 0.5*np.ones_like(data['point'][:,:3]))

geometries = [pcd]

for res in result[0]:
    res_dict = res.to_dict()
    print(f'bounding box {res_dict["label"]} x, y, z, w, h, l, theta:')
    print(res.to_xyzwhlr())

    line_set, box3d = translate_boxes_to_open3d_instance(res.to_xyzwhlr(), res_dict["label"])

    geometries.append(line_set)


o3d.visualization.draw_plotly(geometries)


bounding box Cyclist x, y, z, w, h, l, theta:
[21.41704369  8.16703415 -1.5199275   0.5904038   1.8623445   1.76353049
  1.53019893]
bounding box Car x, y, z, w, h, l, theta:
[17.66450119  8.64189816 -1.52344251  1.63222754  4.06144667  1.57162774
 -3.10736346]
bounding box Car x, y, z, w, h, l, theta:
[12.00894737  7.93859673 -1.38776064  1.65610409  4.05909872  1.55686092
 -2.47253156]
bounding box Car x, y, z, w, h, l, theta:
[16.79777145 13.69311619 -1.79524428  1.67516363  4.03367996  1.49377501
 -2.15967631]
bounding box Car x, y, z, w, h, l, theta:
[60.49819183 -4.68910551 -1.87466407  1.73205876  4.37716055  1.51198149
 -1.54930294]
bounding box Car x, y, z, w, h, l, theta:
[51.78730392  3.02961063 -1.98620945  1.64358139  4.00043631  1.46580136
 -1.59703255]
bounding box Car x, y, z, w, h, l, theta:
[ 52.91270828 -15.26095104  -0.96500158   1.7030915    4.22402811
   2.0738616    1.59574187]
bounding box Car x, y, z, w, h, l, theta:
[ 5.63607025 -5.91612148 -1.59175789  1.6252